In [ ]:
#hide
#default_exp tools.change_manager
from nbdev.showdoc import show_doc

# Change manager

Displays the relationship between scores obtained across experiments with two different evaluation metrics indicated as argument. Calculates Pearson R-squared coefficient.

In [ ]:
#export
import argparse
from importlib import reload
import warnings
import pdb
warnings.filterwarnings('ignore')

def change_manager (subclass=None):
    import hpsearch.config.manager_factory as mf
    reload(mf)
        
    manager_factory = mf.ManagerFactory(verbose=2)
    
    if subclass is None:
        manager_factory.list_subclasses()
    else:
        manager_factory.change_manager(subclass)
        manager_factory.import_written_manager ()
        change_manager (None)

def main():
    parser = argparse.ArgumentParser(description='change experiment manager')
    parser.add_argument('-m','--manager', type=str, default=None, help="new experiment manager to use")
    parser.add_argument('-l', '--list', action= "store_true", help="list experiment managers registered so far")
    pars = parser.parse_args()
    
    #pdb.set_trace()
    if pars.list:
        change_manager (None)
    else:
        if pars.manager is None:
            print ('you did not indicate any manager to change to, listing the managers available')
        change_manager (pars.manager)

In [ ]:
from hpsearch.config.manager_factory import ManagerFactory
manager_factory = ManagerFactory()
manager_factory.delete_and_reset_all()
change_manager (None)

from hpsearch.examples.example_experiment_manager import ExampleExperimentManager
em = ExampleExperimentManager()
manager_factory.register_manager(em)
manager_factory.write_manager(em)

change_manager (None)

current path: /mnt/athena/hpsearch/nbs/tools
subclasses: dict_keys([])
experiment manager registered: ExperimentManager
current path: /mnt/athena/hpsearch/nbs/tools
subclasses: dict_keys(['ExampleExperimentManager'])
experiment manager registered: ExampleExperimentManager


In [ ]:
from hpsearch.examples.dummy_experiment_manager import DummyExperimentManager

em = DummyExperimentManager()
manager_factory.register_manager(em)
manager_factory.write_manager(em)
assert manager_factory.get_experiment_manager().__class__.__name__ == 'DummyExperimentManager'
change_manager (None)

current path: /mnt/athena/hpsearch/nbs/tools
subclasses: dict_keys(['ExampleExperimentManager', 'DummyExperimentManager'])
experiment manager registered: ExperimentManager


In [ ]:
change_manager('ExampleExperimentManager')
assert manager_factory.get_experiment_manager().__class__.__name__ == 'ExampleExperimentManager'

current path: /mnt/athena/hpsearch/nbs/tools
current path: /mnt/athena/hpsearch/nbs/tools
current path: /mnt/athena/hpsearch/nbs/tools
subclasses: dict_keys(['ExampleExperimentManager', 'DummyExperimentManager'])
experiment manager registered: ExampleExperimentManager


In [ ]:
#hide
from nbdev.export import *
notebook2script(recursive=True)

Converted hpconfig.ipynb.
Converted manager_factory.ipynb.
Converted dummy_experiment_manager.ipynb.
Converted example_experiment.ipynb.
Converted example_experiment_manager.ipynb.
Converted experiment_manager.ipynb.
Converted index.ipynb.
Converted change_manager.ipynb.
Converted query.ipynb.
Converted experiment_utils.ipynb.
Converted organize_experiments.ipynb.
Converted metric_correlation.ipynb.
Converted plotly.ipynb.
